## hw_2_2

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [2]:
import pandas as pd

# 데이터 로드
file_path = "customer_data.csv"
df = pd.read_csv(file_path)
df

,CustomerID,Age,AnnualIncome,SpendingScore,PurchaseHistory
0,1,47.0,63.0,19,12
1,2,31.0,73.0,67,7
2,3,NaN,NaN,1,17
3,4,33.0,66.0,71,15
4,5,22.0,127.0,61,11
...,...,...,...,...,...
995,996,63.0,102.0,12,19
996,997,55.0,20.0,11,7
997,998,49.0,102.0,39,5
998,999,21.0,135.0,15,17


In [3]:
# ===== 1. 결측치 확인 =====
print("=== 결측치 개수 확인 ===")
print(df.isna().sum())  # <빈칸>: 각 컬럼별 결측치 개수를 출력하는 코드 작성

# ===== 2. 결측치 처리 =====
# 'Age'와 'AnnualIncome' 컬럼의 결측값을 중앙값(median)으로 대체
df['Age'].fillna(df['Age'].median(), inplace=True)  # <빈칸>: 'Age'의 중앙값으로 결측치 대체
df['AnnualIncome'].fillna(df['AnnualIncome'].median(), inplace=True)  # <빈칸>: 'AnnualIncome'의 중앙값으로 결측치 대체

# 결측치 처리 후 다시 확인
print("\n=== 결측치 처리 후 확인 ===")
print(df.isna().sum())  # <빈칸>: 결측치 개수가 모두 0이 되었는지 확인

=== 결측치 개수 확인 ===
CustomerID           0
Age                100
AnnualIncome       100
SpendingScore        0
PurchaseHistory      0
dtype: int64

=== 결측치 처리 후 확인 ===
CustomerID         0
Age                0
AnnualIncome       0
SpendingScore      0
PurchaseHistory    0
dtype: int64


In [4]:
# ===== 3. 이상치 탐지 (IQR 방식) =====
# IQR(Interquartile Range)을 사용하여 이상치 탐지 및 처리

def detect_outliers_iqr(data, column):
    """
    IQR(사분위 범위) 방법을 이용하여 이상치를 탐지하는 함수
    - IQR = Q3(75%) - Q1(25%)
    - 이상치 기준: Q1 - 1.5*IQR 이하 또는 Q3 + 1.5*IQR 이상
    """
    Q1 = data[column].quantile(.25)  # <빈칸>: 1사분위(25%) 값
    Q3 = data[column].quantile(.75)  # <빈칸>: 3사분위(75%) 값
    IQR = Q3 - Q1  # 사분위 범위 계산
    lower_bound = Q1 - 1.5 * IQR  # <빈칸>: 하한선 (Q1 - 1.5 * IQR)
    upper_bound = Q3 + 1.5 * IQR  # <빈칸>: 상한선 (Q3 + 1.5 * IQR)
    
    # 이상치 데이터 필터링
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers

# 'Age', 'AnnualIncome', 'SpendingScore' 컬럼에서 이상치 탐지
outliers_age = detect_outliers_iqr(df, 'Age')  # <빈칸>: 이상치 탐지 대상 컬럼 입력
outliers_income = detect_outliers_iqr(df, 'AnnualIncome')  # <빈칸>: 이상치 탐지 대상 컬럼 입력
outliers_spending = detect_outliers_iqr(df, 'SpendingScore')  # <빈칸>: 이상치 탐지 대상 컬럼 입력

print("\n=== 이상치 탐지 결과 ===")
print(f"'Age' 이상치 개수: {len(outliers_age)}")  # <빈칸>: 'Age' 컬럼 이상치 개수 출력
print(f"'AnnualIncome' 이상치 개수: {len(outliers_income)}")  # <빈칸>: 'AnnualIncome' 컬럼 이상치 개수 출력
print(f"'SpendingScore' 이상치 개수: {len(outliers_spending)}")  # <빈칸>: 'SpendingScore' 컬럼 이상치 개수 출력


=== 이상치 탐지 결과 ===
'Age' 이상치 개수: 0
'AnnualIncome' 이상치 개수: 0
'SpendingScore' 이상치 개수: 0


In [5]:
# ===== 4. 이상치 처리 =====
# 이상치를 평균값으로 대체 (다른 방법: 삭제 또는 특정 값으로 대체 가능)
df.loc[df['Age'] < (df['Age'].quantile(.25)-1.5*(df['Age'].quantile(.75)-df['Age'].quantile(.25))), 'Age'] = df['Age'].mean()  # <빈칸>: 평균값으로 대체
df.loc[df['Age'] > (df['Age'].quantile(.75)+1.5*(df['Age'].quantile(.75)-df['Age'].quantile(.25))), 'Age'] = df['Age'].mean()  # <빈칸>: 평균값으로 대체

df.loc[df['AnnualIncome'] < (df['AnnualIncome'].quantile(.25)-1.5*(df['AnnualIncome'].quantile(.75)-df['AnnualIncome'].quantile(.25))), 'AnnualIncome'] = df['AnnualIncome'].mean()  # <빈칸>: 평균값으로 대체
df.loc[df['AnnualIncome'] > (df['AnnualIncome'].quantile(.75)-1.5*(df['AnnualIncome'].quantile(.75)-df['AnnualIncome'].quantile(.25))), 'AnnualIncome'] = df['AnnualIncome'].mean()  # <빈칸>: 평균값으로 대체

df.loc[df['SpendingScore'] < (df['SpendingScore'].quantile(.25)-1.5*(df['SpendingScore'].quantile(.75)-df['SpendingScore'].quantile(.25))), 'SpendingScore'] = df['SpendingScore'].mean()  # <빈칸>: 평균값으로 대체
df.loc[df['SpendingScore'] > (df['SpendingScore'].quantile(.75)+1.5*(df['SpendingScore'].quantile(.75)-df['SpendingScore'].quantile(.25))), 'SpendingScore'] = df['SpendingScore'].mean()  # <빈칸>: 평균값으로 대체

# 이상치 처리 후 다시 확인
print("\n=== 이상치 처리 후 확인 ===")
print(f"최소/최대 Age: {df['Age'].min()} ~ {df['Age'].max()}")  # <빈칸>: 최소/최대 값 출력
print(f"최소/최대 AnnualIncome: {df['AnnualIncome'].min()} ~ {df['AnnualIncome'].max()}")  # <빈칸>: 최소/최대 값 출력
print(f"최소/최대 SpendingScore: {df['SpendingScore'].min()} ~ {df['SpendingScore'].max()}")  # <빈칸>: 최소/최대 값 출력



=== 이상치 처리 후 확인 ===
최소/최대 Age: 18.0 ~ 69.0
최소/최대 AnnualIncome: 20.0 ~ 84.541
최소/최대 SpendingScore: 1.0 ~ 100.0


## hw_2_4

In [6]:
# =============================================================================
# merge() 함수 설명:
# - df_purchase: 구매 데이터
# - df_demo: 고객 인구통계 데이터
# - on="CustomerID": 'CustomerID'를 기준으로 두 데이터를 병합
# - how="inner": 공통되는 'CustomerID' 값만 포함 (교집합)
#  df = pd.merge(df_purchase, df_demo, on="CustomerID", how="inner")
# =============================================================================
# =============================================================================
# groupby() 함수 설명:
# - "CustomerID"를 기준으로 데이터를 그룹화
# - 각 고객별 'PurchaseAmount' 값을 합산하여 총 소비 금액(TotalSpent) 계산
# - reset_index(): 그룹화된 데이터를 다시 일반 데이터프레임 형태로 변환
#  df_total_spent = df.groupby("CustomerID")["PurchaseAmount"].sum().reset_index()
# =============================================================================
# =============================================================================
# cut() 함수 설명:
# - df["Age"]: 나이(Age) 데이터를 기반으로 연령대를 구분
# - bins: 연령 구간 설정 (0-18, 19-30, 31-50, 51-100)
# - labels: 각 구간에 할당할 라벨(Teen, Young Adult, Adult, Senior)
# - right=False: 오른쪽 경계값(예: 18)은 포함하지 않도록 설정 (0 ≤ x < 18은 Teen)
#  df["AgeGroup"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)
# =============================================================================
# =============================================================================
# map() 함수 설명:
# - df["Gender"]: 기존 성별 데이터 (0 또는 1)
# - .map({0: "Female", 1: "Male"}): 0 → "Female", 1 → "Male"로 변환
# =============================================================================


# 1. 필요한 라이브러리 불러오기
import pandas as pd

# 2. 데이터 로드
purchase_file_path = "purchase_data.csv"
demo_file_path = "demo_data.csv"

df_purchase = pd.read_csv(purchase_file_path)  # <빈칸>: 구매 데이터 로드
df_demo = pd.read_csv(demo_file_path)  # <빈칸>: 고객 정보 데이터 로드

# 3. 데이터 확인
print("=== 구매 데이터 확인 ===")
df_purchase.head()  # <빈칸>: 구매 데이터의 상위 5개 행 출력

=== 구매 데이터 확인 ===


,CustomerID,PurchaseAmount,PurchaseDate
0,1,44,2023-01-01
1,2,231,2023-01-02
2,3,68,2023-01-03
3,4,177,2023-01-04
4,5,288,2023-01-05


In [8]:
print("\n=== 고객 정보 데이터 확인 ===")
df_demo.head()  # <빈칸>: 고객 정보 데이터의 상위 5개 행 출력


=== 고객 정보 데이터 확인 ===


,CustomerID,Gender,Age,AnnualIncome
0,1,1,60,119
1,2,0,18,41
2,3,1,27,124
3,4,1,23,24
4,5,1,50,108


In [9]:
# 4. 데이터 병합 (CustomerID 기준)
# 두 데이터프레임을 CustomerID를 기준으로 병합 (Inner Join)
df = pd.merge(df_purchase, df_demo, on="CustomerID", how="inner")  # <빈칸>: 병합

# 병합 후 데이터 확인
print("\n=== 병합된 데이터 확인 ===")
df.head()  # <빈칸>: 병합된 데이터의 상위 5개 행 출력


=== 병합된 데이터 확인 ===


,CustomerID,PurchaseAmount,PurchaseDate,Gender,Age,AnnualIncome
0,1,44,2023-01-01,1,60,119
1,2,231,2023-01-02,0,18,41
2,3,68,2023-01-03,1,27,124
3,4,177,2023-01-04,1,23,24
4,5,288,2023-01-05,1,50,108


In [11]:
# 5. 데이터 타입 변환
# 'PurchaseDate' 컬럼을 문자열에서 datetime 타입으로 변환
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])  # <빈칸>: 날짜 형식 변환

# 변환된 데이터 타입 확인
print("\n=== 데이터 타입 확인 ===")
df.info()  # <빈칸>: 데이터 타입 확인


=== 데이터 타입 확인 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CustomerID      1000 non-null   int64         
 1   PurchaseAmount  1000 non-null   int64         
 2   PurchaseDate    1000 non-null   datetime64[ns]
 3   Gender          1000 non-null   int64         
 4   Age             1000 non-null   int64         
 5   AnnualIncome    1000 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 47.0 KB


In [17]:
# 6. 고객별 총 구매 금액 계산
# 'CustomerID'를 기준으로 'PurchaseAmount'를 합산하여 고객별 총 구매 금액 계산
df_total_spent = df.groupby("CustomerID")["PurchaseAmount"].sum().reset_index()  # <빈칸>: 고객별 총 구매 금액 계산

# 컬럼명 변경 (총 구매 금액을 의미하도록 변경)
df_total_spent.rename(columns={"PurchaseAmount": "TotalSpent"}, inplace=True)  # <빈칸>: 컬럼명 변경

# 고객별 총 구매 금액 확인
print("\n=== 고객별 총 구매 금액 확인 ===")
df_total_spent.head()  # <빈칸>: 상위 5개 데이터 출력


=== 고객별 총 구매 금액 확인 ===


,CustomerID,TotalSpent
0,1,44
1,2,231
2,3,68
3,4,177
4,5,288


In [18]:
# 7. 연령대별 평균 소비 금액 계산
# 연령대를 나누는 기준을 설정 (Pandas의 cut() 사용)
bins = [0, 19, 31, 51, 100]  # 연령대 구간 (0-18세, 19-30세, 31-50세, 51세 이상)
labels = ["Teen", "Young Adult", "Adult", "Senior"]  # 각 구간에 대한 라벨 지정
df["AgeGroup"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)  # <빈칸>: 연령대 구분

# 연령대별 평균 소비 금액 계산
df_age_spent = df.groupby("AgeGroup")["PurchaseAmount"].mean().reset_index()  # <빈칸>: 연령대별 평균 소비 금액 계산

# 컬럼명 변경
df_age_spent.rename(columns={"PurchaseAmount": "AvgSpent"}, inplace=True)  # <빈칸>: 컬럼명 변경

# 연령대별 평균 소비 금액 확인
print("\n=== 연령대별 평균 소비 금액 확인 ===")
df_age_spent  # 연령대별 평균 소비 금액 출력


=== 연령대별 평균 소비 금액 확인 ===


,AgeGroup,AvgSpent
0,Teen,121.266667
1,Young Adult,126.385542
2,Adult,123.115385
3,Senior,120.715054


In [19]:
# 8. 성별별 평균 소비 금액 계산
# 성별(Gender)별 평균 소비 금액 계산
df_gender_spent = df.groupby("Gender")["PurchaseAmount"].mean().reset_index()  # <빈칸>: 성별별 평균 소비 금액 계산

# Gender 값을 남/녀로 변환 (0: 여성, 1: 남성)
df_gender_spent["Gender"] = df_gender_spent["Gender"].map({0: "Female", 1: "Male"})  # <빈칸>: 성별 변환

# 컬럼명 변경
df_gender_spent.rename(columns={"PurchaseAmount": "AvgSpent"}, inplace=True)  # <빈칸>: 컬럼명 변경

# 성별별 평균 소비 금액 확인
print("\n=== 성별별 평균 소비 금액 확인 ===")
df_gender_spent  # 성별별 평균 소비 금액 출력


=== 성별별 평균 소비 금액 확인 ===


,Gender,AvgSpent
0,Female,124.646707
1,Male,121.364729


In [20]:
# 9. 고객별 총 구매 횟수 계산
# CustomerID를 기준으로 구매 횟수(= 데이터 개수) 계산
df_purchase_count = df.groupby("CustomerID")["PurchaseAmount"].count().reset_index()  # <빈칸>: 고객별 총 구매 횟수 계산

# 컬럼명 변경 (총 구매 횟수를 의미하도록 변경)
df_purchase_count.rename(columns={"PurchaseAmount": "TotalPurchases"}, inplace=True)  # <빈칸>: 컬럼명 변경

# 고객별 총 구매 횟수 확인
print("\n=== 고객별 총 구매 횟수 확인 ===")
df_purchase_count.head()  # <빈칸>: 상위 5개 데이터 출력


=== 고객별 총 구매 횟수 확인 ===


,CustomerID,TotalPurchases
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [23]:
# 10. 최종 데이터 병합 (총 소비 금액 + 총 구매 횟수)
df_final = pd.merge(df_total_spent, df_purchase_count, on="CustomerID", how="inner")  # <빈칸>: 최종 데이터 병합

# 최종 데이터 확인
print("\n=== 고객별 총 소비 금액 및 총 구매 횟수 데이터 확인 ===")
df_final.value_counts()  # <빈칸>: 최종 데이터 출력



=== 고객별 총 소비 금액 및 총 구매 횟수 데이터 확인 ===


CustomerID  TotalSpent  TotalPurchases
1000        36          1                 1
1           44          1                 1
2           231         1                 1
3           68          1                 1
4           177         1                 1
                                         ..
13          42          1                 1
12          24          1                 1
11          45          1                 1
10          188         1                 1
9           7           1                 1
Name: count, Length: 1000, dtype: int64